In [76]:
#Needed to successfully import as a wordcould
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from collections import defaultdict
import json


In [77]:
json_list=[]
trump=pd.read_csv("../data/trump_tweets_classified.csv")
trump.head()
trump.count()

Unnamed: 0      86
Unnamed: 0.1    86
id              86
Created at      86
Screen Name     86
Tweet Text      86
sentiment       86
dtype: int64

In [78]:
trump2=trump[["id", "Created at", "Screen Name", "Tweet Text", "sentiment"]]
trump2.count()

id             86
Created at     86
Screen Name    86
Tweet Text     86
sentiment      86
dtype: int64

In [79]:
trump2["Frequency"]=1

/Users/dhanyamaheswaran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
#modifying the date-time string 
time_string=trump2["Created at"].to_list()
time_edited=[]
for item in time_string:
    item2=item.replace(" +0000","")
    time_edited.append(item2)
    

In [81]:
trump2["Created at"]=time_edited
trump2['Created at'] =  pd.to_datetime(trump2['Created at'], infer_datetime_format=True)

/Users/dhanyamaheswaran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/dhanyamaheswaran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [82]:
trump2['date'] = trump2['Created at'].dt.date
#most frequent dates most are from the same day, wil lscreq axis if not all from the same day 
date = pd.DataFrame(trump2.groupby("date")["Frequency"].count())
date=date.sort_values(by='Frequency', ascending=False)
#to keep majority of the graph, only display one day of data
date=date.head(1)
day=date.index.values[0]



In [83]:
edited_time_df=trump2.loc[trump2["date"]==day, ["date", "Created at", "Screen Name", "Tweet Text", "sentiment", "Frequency"]]
edited=edited_time_df.sort_values(by='Created at')
edited=edited.rename(columns={"Created at": "datetime"})
edited=edited.drop(columns=['date'])

In [84]:
#json positive sentiment
only_positive = edited.loc[edited["sentiment"] == "positive", :]
positive_frequency=only_positive[["datetime", "Frequency"]]
only_positive=positive_frequency.resample('0.05T', on='datetime').sum()
only_positive=only_positive.reset_index()

In [85]:
only_positive["sentiment"]="positive"
only_positive

,datetime,Frequency,sentiment
0,2020-10-30 03:00:15,27,positive
1,2020-10-30 03:00:18,1,positive


In [86]:
#json positive sentiment
only_negative = edited.loc[edited["sentiment"] == "negative", :]
negative_frequency=only_negative[["datetime", "Frequency"]]
only_negative=negative_frequency.resample('0.05T', on='datetime').sum()
only_negative=only_negative.reset_index()

In [87]:
only_negative["sentiment"]="negative"
only_negative

,datetime,Frequency,sentiment
0,2020-10-30 03:00:15,54,negative
1,2020-10-30 03:00:18,1,negative


In [88]:
frames=[only_positive, only_negative]
sentiment_concat=pd.concat(frames)
sentiment_concat['datetime']=sentiment_concat['datetime'].astype(str)

In [89]:
sentiment_group = sentiment_concat.groupby(['sentiment', "datetime"])
sentiment_group_2=pd.DataFrame(sentiment_group["Frequency"].sum())
sentiment_group_2


Frequency
sentiment datetime                      
negative  2020-10-30 03:00:15         54
          2020-10-30 03:00:18          1
positive  2020-10-30 03:00:15         27
          2020-10-30 03:00:18          1

In [90]:
results1 = defaultdict(lambda: defaultdict(dict))

for index, value in sentiment_group_2.itertuples():
    for i, key in enumerate(index):
        if i == 0:
            nested = results1[key]
        elif i == len(index) - 1:
            nested[key] = value
        else:
            nested = nested[key] 

In [91]:
json_list.append(results1)
json_list

[defaultdict(<function __main__.<lambda>()>,
             {'negative': defaultdict(dict,
                          {'2020-10-30 03:00:15': 54,
                           '2020-10-30 03:00:18': 1}),
              'positive': defaultdict(dict,
                          {'2020-10-30 03:00:15': 27,
                           '2020-10-30 03:00:18': 1})})]

In [92]:
#obtaining panel 
sentiment = trump2.groupby("sentiment")["Frequency"].count()
total_sentiment=trump2["sentiment"].count()
percentage_sentiment=((sentiment/total_sentiment)*100).round()

In [93]:
percentage_sentiment


sentiment
negative    65.0
positive    35.0
Name: Frequency, dtype: float64

In [94]:

percentage_dict=percentage_sentiment.to_dict()
json_list.append(percentage_dict)


In [95]:
example=trump2.head(5)

    
   
   

In [96]:
screen_name=example["Screen Name"].to_list()
tweet_text=example["Tweet Text"].to_list()


In [97]:
 res = {screen_name[i]: tweet_text[i] for i in range(len(screen_name))}

trump_dict={"trump_tweets":res}


In [98]:
 json_list.append(trump_dict)

In [99]:
with open('twitter_data.json', 'w') as fp:
    json.dump(json_list, fp)

In [100]:
json_list

[defaultdict(<function __main__.<lambda>()>,
             {'negative': defaultdict(dict,
                          {'2020-10-30 03:00:15': 54,
                           '2020-10-30 03:00:18': 1}),
              'positive': defaultdict(dict,
                          {'2020-10-30 03:00:15': 27,
                           '2020-10-30 03:00:18': 1})}),
 {'negative': 65.0, 'positive': 35.0},
 {'trump_tweets': {'RealJamesWoods': 'This smug little number saying her company, #Google, shouldn’t be broken up. Why? Because smaller companies won’t h… https://t.co/XCxmoS4YTc',
   'SethAbramson': "As good an ad about Trump as I've seen. I hope you'll retweet it. https://t.co/G5e5L0HyRu",
   'PeteButtigieg': "Trump's rallies are a metaphor for how he views his own supporters. Happy to have them cheer for him, only to leave… https://t.co/hKnjHluBUo",
   'bflyjesusgrl': 'RT @TeamTrump: WATCH: @IngrahamAngle on The GREAT Trump Recovery! https://t.co/j6P8aix12N',
   'um_cloudy': '@censors68769010 Chris